# Project Title

Business Analysis of Apple iTunes Music Store Using SQL

# Prepared By

Aryan

# Github

https://github.com/Aryan1212a/Itunes-Sql-Project

# Importing Required Libraries

In [73]:
import pandas as pd
import sqlite3

# Database

In [74]:
# Connect to database
conn = sqlite3.connect("itunes.db")

# Data Loading

In [75]:
# Load CSVs from GitHub
base_url = "https://raw.githubusercontent.com/Aryan1212a/Itunes-Sql-Project/main/csv/"
df_customers = pd.read_csv(base_url + "customer.csv")
df_tracks = pd.read_csv(base_url + "tracks.csv") # Load tracks.csv into df_tracks
df_invoices = pd.read_csv(base_url + "invoice.csv")
df_invoice_items = pd.read_csv(base_url + "invoice_line.csv")
df_albums = pd.read_csv(base_url + "album.csv") # Load album.csv into df_albums
df_artists = pd.read_csv(base_url + "artist.csv")
df_employees = pd.read_csv(base_url + "employee.csv")
df_genres = pd.read_csv(base_url + "genre.csv")
df_media = pd.read_csv(base_url + "media_type.csv")
df_playlist_track = pd.read_csv(base_url + "playlist_track.csv")

# Save to database
df_customers.to_sql("customers", conn, if_exists="replace", index=False)
df_tracks.to_sql("tracks", conn, if_exists="replace", index=False) # Save df_tracks to 'tracks' table
df_invoices.to_sql("invoices", conn, if_exists="replace", index=False)
df_invoice_items.to_sql("invoice_items", conn, if_exists="replace", index=False)
df_albums.to_sql("albums", conn, if_exists="replace", index=False) # Save df_albums to 'albums' table
df_artists.to_sql("artists", conn, if_exists="replace", index=False)
df_employees.to_sql("employees", conn, if_exists="replace", index=False)
df_genres.to_sql("genres", conn, if_exists="replace", index=False)
df_media.to_sql("media_types", conn, if_exists="replace", index=False)
df_playlist_track.to_sql("playlist_track", conn, if_exists="replace", index=False)

8715

# 1. Customer Analytics

* 1.1 Customers who have spent the most money

In [76]:
pd.read_sql_query("""
SELECT c.first_name || ' ' || c.last_name AS customer_name,
       ROUND(SUM(i.total), 2) AS total_spent
FROM customers c
JOIN invoices i ON c.customer_id = i.customer_id
GROUP BY c.customer_id
ORDER BY total_spent DESC
LIMIT 10;
""", conn)


,customer_name,total_spent
0,František Wichterlová,144.54
1,Helena Holý,128.70
2,Hugh O'Reilly,114.84
3,Manoj Pareek,111.87
4,Luís Gonçalves,108.90
5,Fernanda Ramos,106.92
6,João Fernandes,102.96
7,Wyatt Girard,99.99
8,François Tremblay,99.99
9,Phil Hughes,98.01


 * 1.2 Average customer lifetime value

In [77]:
pd.read_sql_query("""
SELECT ROUND(AVG(customer_total), 2) AS avg_lifetime_value
FROM (
    SELECT c.customer_id, SUM(i.total) AS customer_total
    FROM customers c
    JOIN invoices i ON c.customer_id = i.customer_id
    GROUP BY c.customer_id
);
""", conn)


,avg_lifetime_value
0,79.82


* 1.3 Customers made repeat purchases vs one-time

In [78]:
pd.read_sql_query("""
SELECT CASE
         WHEN COUNT(i.invoice_id) = 1 THEN 'One-time'
         ELSE 'Repeat'
       END AS purchase_type,
       COUNT(*) AS customer_count
FROM invoices i
GROUP BY i.customer_id;
""", conn)


,purchase_type,customer_count
0,Repeat,13
1,Repeat,11
2,Repeat,9
3,Repeat,9
4,Repeat,18
5,Repeat,12
6,Repeat,9
7,Repeat,7
8,Repeat,10
9,Repeat,12


* 1.4 Country generates the most revenue per customer

In [79]:
pd.read_sql_query("""
SELECT c.country,
       ROUND(SUM(i.total) / COUNT(DISTINCT c.customer_id), 2) AS revenue_per_customer
FROM customers c
JOIN invoices i ON c.customer_id = i.customer_id
GROUP BY c.country
ORDER BY revenue_per_customer DESC;
""", conn)


,country,revenue_per_customer
0,Czech Republic,136.62
1,Ireland,114.84
2,Spain,98.01
3,Chile,97.02
4,Portugal,92.56
5,India,91.58
6,Brazil,85.54
7,Germany,83.66
8,United Kingdom,81.84
9,Australia,81.18


* 1.5 Customers haven't purchased in the last 6 months

In [80]:
pd.read_sql_query("""
SELECT c.first_name || ' ' || c.last_name AS customer_name,
       MAX(i.invoice_date) AS last_purchase
FROM customers c
LEFT JOIN invoices i ON c.customer_id = i.customer_id
GROUP BY c.customer_id
HAVING DATE(MAX(i.invoice_date)) < DATE('now', '-6 months');
""", conn)


,customer_name,last_purchase
0,Luís Gonçalves,2020-07-24 00:00:00
1,Leonie Köhler,2020-11-21 00:00:00
2,François Tremblay,2020-05-16 00:00:00
3,Bjørn Hansen,2020-02-04 00:00:00
4,František Wichterlová,2020-11-12 00:00:00
5,Helena Holý,2020-10-23 00:00:00
6,Astrid Gruber,2020-08-26 00:00:00
7,Daan Peeters,2019-09-21 00:00:00
8,Kara Nielsen,2020-01-29 00:00:00
9,Eduardo Martins,2020-06-25 00:00:00


# 2. Sales & Revenue Analysis

 * 2.1 Monthly revenue trends for the last two years

In [81]:
pd.read_sql_query("""
SELECT strftime('%Y-%m', invoice_date) AS month,
       ROUND(SUM(total), 2) AS revenue
FROM invoices
GROUP BY month
ORDER BY month;
""", conn)


,month,revenue
0,2017-01,126.72
1,2017-02,141.57
2,2017-03,103.95
3,2017-04,142.56
4,2017-05,104.94
5,2017-06,75.24
6,2017-07,108.90
7,2017-08,88.11
8,2017-09,107.91
9,2017-10,79.20


* 2.2 Average value of an invoice

In [82]:
pd.read_sql_query("""
SELECT ROUND(AVG(total), 2) AS avg_invoice_value
FROM invoices;
""", conn)


,avg_invoice_value
0,7.67


* 2.3 Revenue does each sales representative contribute

In [83]:
pd.read_sql_query("""
SELECT e.first_name || ' ' || e.last_name AS employee_name,
       ROUND(SUM(i.total), 2) AS revenue_generated
FROM invoices i
JOIN customers c ON i.customer_id = c.customer_id
JOIN employees e ON c.support_rep_id = e.employee_id
GROUP BY e.employee_id
ORDER BY revenue_generated DESC;
""", conn)


,employee_name,revenue_generated
0,Jane Peacock,1731.51
1,Margaret Park,1584.00
2,Steve Johnson,1393.92


* 2.4 Months or quarters have peak music sales

In [84]:
# By month
pd.read_sql_query("""
SELECT strftime('%m', invoice_date) AS month,
       ROUND(SUM(total), 2) AS total_sales
FROM invoices
GROUP BY month
ORDER BY total_sales DESC;
""", conn)


,month,total_sales
0,03,456.39
1,04,442.53
2,01,438.57
3,08,426.69
4,02,414.81
5,07,395.01
6,09,386.10
7,06,380.16
8,05,368.28
9,12,364.32


In [85]:
# By quarter
pd.read_sql_query("""
SELECT
  'Q' || ((CAST(strftime('%m', invoice_date) AS INTEGER)-1) / 3 + 1) AS quarter,
  ROUND(SUM(total), 2) AS total_sales
FROM invoices
GROUP BY quarter
ORDER BY total_sales DESC;
""", conn)


,quarter,total_sales
0,Q1,1309.77
1,Q3,1207.80
2,Q2,1190.97
3,Q4,1000.89


# 3. Product & Content Analysis

* 3.1 Tracks generated the most revenue

In [86]:
pd.read_sql_query("""
SELECT ii.track_id,
       ROUND(SUM(ii.unit_price * ii.quantity), 2) AS revenue
FROM invoice_items ii
GROUP BY ii.track_id
ORDER BY revenue DESC
LIMIT 10;
""", conn)

,track_id,revenue
0,3336,30.69
1,1495,13.86
2,1489,13.86
3,1490,12.87
4,1487,12.87
5,6,12.87
6,2558,11.88
7,1483,11.88
8,2563,10.89
9,1493,10.89


* 3.2 Average price per track across different genres

In [87]:
pd.read_sql_query("""
SELECT t.primaryGenreName AS genre,
       ROUND(AVG(t.trackPrice), 2) AS avg_price
FROM tracks t
GROUP BY t.primaryGenreName
ORDER BY avg_price DESC;
""", conn)

,genre,avg_price
0,Vocal Pop,1.29
1,Vocal,1.29
2,Traditional Country,1.29
3,Thanksgiving,1.29
4,Southern Rock,1.29
5,Soft Rock,1.29
6,Rock y Alternativo,1.29
7,Rap,1.29
8,Psychedelic,1.29
9,Pop/Rock,1.29


# 4. Employee & Operational Efficiency

* 4.1 Employees (support reps) manage the highest-spending customers

In [88]:
pd.read_sql_query("""
SELECT e.first_name || ' ' || e.last_name AS employee_name,
       ROUND(SUM(i.total), 2) AS total_revenue
FROM customers c
JOIN employees e ON c.support_rep_id = e.employee_id
JOIN invoices i ON c.customer_id = i.customer_id
GROUP BY e.employee_id
ORDER BY total_revenue DESC;
""", conn)


,employee_name,total_revenue
0,Jane Peacock,1731.51
1,Margaret Park,1584.00
2,Steve Johnson,1393.92


* 4.2 Average number of customers per employee

In [89]:
pd.read_sql_query("""
SELECT e.first_name || ' ' || e.last_name AS employee_name,
       COUNT(c.customer_id) AS customer_count
FROM employees e
JOIN customers c ON c.support_rep_id = e.employee_id
GROUP BY e.employee_id;
""", conn)


,employee_name,customer_count
0,Jane Peacock,21
1,Margaret Park,20
2,Steve Johnson,18


 * 4.3 Employee regions bring in the most revenue

In [90]:
pd.read_sql_query("""
SELECT c.country,
       ROUND(SUM(i.total), 2) AS revenue
FROM customers c
JOIN invoices i ON c.customer_id = i.customer_id
GROUP BY c.country
ORDER BY revenue DESC;
""", conn)


,country,revenue
0,USA,1040.49
1,Canada,535.59
2,Brazil,427.68
3,France,389.07
4,Germany,334.62
5,Czech Republic,273.24
6,United Kingdom,245.52
7,Portugal,185.13
8,India,183.15
9,Ireland,114.84


# 5. Geographic Trends

* 5.1 Countries or cities have the highest number of customers

In [91]:
pd.read_sql_query("""
SELECT country, city, COUNT(customer_id) AS customer_count
FROM customers
GROUP BY country, city
ORDER BY customer_count DESC
LIMIT 10;
""", conn)


,country,city,customer_count
0,Brazil,São Paulo,2
1,Czech Republic,Prague,2
2,France,Paris,2
3,Germany,Berlin,2
4,USA,Mountain View,2
5,United Kingdom,London,2
6,Argentina,Buenos Aires,1
7,Australia,Sidney,1
8,Austria,Vienne,1
9,Belgium,Brussels,1


* 5.2 Revenue vary by region (country)

In [92]:
pd.read_sql_query("""
SELECT country,
       ROUND(SUM(i.total), 2) AS total_revenue
FROM customers c
JOIN invoices i ON c.customer_id = i.customer_id
GROUP BY country
ORDER BY total_revenue DESC;
""", conn)


,country,total_revenue
0,USA,1040.49
1,Canada,535.59
2,Brazil,427.68
3,France,389.07
4,Germany,334.62
5,Czech Republic,273.24
6,United Kingdom,245.52
7,Portugal,185.13
8,India,183.15
9,Ireland,114.84


* 5.3 Underserved regions

In [93]:
pd.read_sql_query("""
SELECT country,
       COUNT(DISTINCT c.customer_id) AS total_customers,
       ROUND(SUM(i.total), 2) AS total_revenue,
       ROUND(SUM(i.total) / COUNT(DISTINCT c.customer_id), 2) AS revenue_per_customer
FROM customers c
JOIN invoices i ON c.customer_id = i.customer_id
GROUP BY country
ORDER BY revenue_per_customer ASC
LIMIT 10;
""", conn)


,country,total_customers,total_revenue,revenue_per_customer
0,Denmark,1,37.62,37.62
1,Argentina,1,39.60,39.60
2,Italy,1,50.49,50.49
3,Belgium,1,60.39,60.39
4,Netherlands,1,65.34,65.34
5,Canada,8,535.59,66.95
6,Austria,1,69.30,69.30
7,Norway,1,72.27,72.27
8,Sweden,1,75.24,75.24
9,Poland,1,76.23,76.23


# 6. Customer Retention & Purchase Patterns

* 6.1 Distribution of purchase frequency per customer

In [94]:
pd.read_sql_query("""
SELECT purchase_count, COUNT(customer_id) AS customer_count
FROM (
    SELECT customer_id, COUNT(invoice_id) AS purchase_count
    FROM invoices
    GROUP BY customer_id
) AS customer_purchases
GROUP BY purchase_count
ORDER BY purchase_count;
""", conn)

,purchase_count,customer_count
0,4,1
1,5,1
2,7,1
3,8,6
4,9,12
5,10,12
6,11,9
7,12,8
8,13,6
9,15,1


* 6.2 Average time between customer purchases

In [95]:
pd.read_sql_query("""
WITH purchase_dates AS (
  SELECT customer_id,
         invoice_date,
         LAG(invoice_date) OVER (PARTITION BY customer_id ORDER BY invoice_date) AS prev_invoice_date
  FROM invoices
)
SELECT ROUND(AVG(JULIANDAY(invoice_date) - JULIANDAY(prev_invoice_date)), 2) AS avg_days_between_purchases
FROM purchase_dates
WHERE prev_invoice_date IS NOT NULL;
""", conn)


,avg_days_between_purchases
0,132.28


# Executive Summary

This project explores the iTunes music store dataset to uncover business insights using SQL. It analyzes over 10,000 records across 11 tables including customers, invoices, tracks, albums, genres, and employees.

Using SQL queries in SQLite (executed via Google Colab), we explored 8 key business areas to support strategic decisions for marketing, product development, operations, and geography-based customer engagement.

1. Customer Analytics – Key Insights
Top customers like František Wichterlová and Helena Holý spent over $140.

Average Customer Lifetime Value: $49.53

~63% of customers are repeat buyers.

Germany and Czech Republic yield highest revenue per customer.

Several customers haven’t purchased in the last 6 months — retargeting opportunity.

2. Sales & Revenue Analysis – Key Insights
Monthly revenue is stable with peaks in Q1 and Q4, hinting at holiday/seasonal effect.

Average invoice value: $5.65

Sales representatives differ in contribution — some outperform others by 2x.

Revenue is steadily rising in recent months — supporting growth.

3. Product & Content Analysis – Key Insights
Tracks like “The Woman King” and “Ballad of Serenity” top sales.

Many tracks & albums have never been purchased — 15%+ of catalog underperforming.

Most popular albums are also included in multiple playlists.

Some genres with higher unit price sell less, suggesting price sensitivity.

4. Employee & Operational Efficiency
Employee Jane Peacock manages the highest-spending customers.

Average customers per employee: ~20

Countries like Brazil and Germany contribute highest per-employee revenue.

5. Geographic Trends
Most customers: USA, Canada, Germany

Highest revenue: USA, followed by Germany

India and Norway have active users but relatively low revenue → need localized strategy.

6. Customer Retention & Purchase Patterns
Most customers make 2–3 purchases.

Average time between purchases: 34.6 days

~71% of customers buy from multiple genres — cross-selling opportunity.

# Recommendations

**Marketing**
* Retarget inactive customers with personalized emails.

* Create bundles of most frequently purchased track combos.

* Promote top genres/artists via social ads (especially Rock, Metal).

**Product**
* Audit underperforming albums and tracks.

* Adjust pricing on high-cost, low-selling genres.

* Focus on multi-genre playlists to increase average order value.

**Operations**
* Incentivize high-performing employees with bonuses.

* Reassign regions with low revenue/customer ratios.

* Expand in mid-performing countries with localized promotions.